In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

100%|██████████| 26421880/26421880 [00:01<00:00, 19150410.73it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 330647.84it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 6115230.25it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 25254125.14it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [3]:
#can also do hyperparameter tuning
learning_rate = 1e-3
batch_size = 64
epochs = 5 #num times to iterate over dataset

#each epoch consists of
  # Train Loop - try to converge on optimal parameters
  # Validation/Test Loop - check validation set to see if model improving

In [4]:
# MSE for regression and NLL (negative Log Likelihood) for clasification
# CrossEntropyLoss combines LogSoftmax and NLL
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

In [6]:
#using Stochastic Gradient Descent
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

#in training loop, zero out the gradients, call .backward() to get gradients, then adjust the parameters by the gradients collected

In [7]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.302752  [   64/60000]
loss: 2.291954  [ 6464/60000]
loss: 2.267657  [12864/60000]
loss: 2.256161  [19264/60000]
loss: 2.254296  [25664/60000]
loss: 2.216076  [32064/60000]
loss: 2.221905  [38464/60000]
loss: 2.193762  [44864/60000]
loss: 2.191798  [51264/60000]
loss: 2.148984  [57664/60000]
Test Error: 
 Accuracy: 47.4%, Avg loss: 2.146478 

Epoch 2
-------------------------------
loss: 2.166702  [   64/60000]
loss: 2.150279  [ 6464/60000]
loss: 2.092329  [12864/60000]
loss: 2.102683  [19264/60000]
loss: 2.062860  [25664/60000]
loss: 2.000823  [32064/60000]
loss: 2.019906  [38464/60000]
loss: 1.950799  [44864/60000]
loss: 1.953013  [51264/60000]
loss: 1.868470  [57664/60000]
Test Error: 
 Accuracy: 57.4%, Avg loss: 1.874230 

Epoch 3
-------------------------------
loss: 1.913184  [   64/60000]
loss: 1.877058  [ 6464/60000]
loss: 1.762698  [12864/60000]
loss: 1.800088  [19264/60000]
loss: 1.696849  [25664/60000]
loss: 1.643741  [32064/600